<a href="https://colab.research.google.com/github/New-AutoMotive/learning/blob/main/BigQuery%20tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BigQuery & Python Tutorial

BigQuery is accessible from https://console.cloud.google.com.  It is a piece of software that allows you to access and interact with large datasets.

## Importing packages and authenticating our access to BigQuery

In [1]:
# To interact with BigQuery, it's helpful to use some python packages.
# The %load_ext command is a variation on the 'import' command, but it only works in Jupyter Notebooks. 
import pandas as pd
%load_ext google.cloud.bigquery

In [ ]:
# When running this notebook in Google Colab, use the following code to authenticate

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

## Running a Query

Below, we query a database with a SQL query, which will be stored in the variable `df`. Since this code is preceded by a % or %% symbol, it is known as a 'cell magic'. The code is not actually python, so these % symbols tell the python interpreter that we are using a different language. 

Remember: 


*   `SELECT` determines the columns we want,
*    `*` is the wildcard character - it denotes any and every possible value available,
*   `FROM` tells the SQL interpreter where to look for the data.
*   `WHERE` imposes filters or conditions on the data that is returned, in this case any row that has monthOfFirstRegistration that satisfies the value. 
*   `AND` adds another filter or condition in addition to the first condition specified after the `WHERE` clause. 







In [3]:
%%bigquery df --project rugged-baton-283921
SELECT *
FROM VES_dataset.VRN_UK
WHERE monthOfFirstRegistration = '2021-12-01'
AND fuelType = 'ELECTRICITY'

## Investigating our data

In [ ]:
# The .head() method shows us the top 5 lines of the spreadsheet. 
df.head()

In [ ]:
df.describe()

In [ ]:
df.fuelType

In [ ]:
df.fuelType.unique()

In [ ]:
df.dtypes

## Investigating New Van Registrations

Now let's set ourselves a data analysis task. At New AutoMotive we tend to focus on passenger cars - but what about the new **van** market? How is that changing? Is diesel dying there, too? Are BEVs common? Let's have a look and see how many Light Commercial Vehicles (with 'N1' type approval) have been registered in each month of the last 5 years, broken down by fuel type. 

### How would we write this SQL query?

In [64]:
%%bigquery df_van --project rugged-baton-283921
# We want three columns: monthOfRegistration, fuelType, and the total number of vans falling into each combination of those two values.
SELECT monthOfFirstRegistration, fuelType, COUNT(*) as total
# The data is coming from VRN_UK
FROM VES_dataset.VRN_UK
# Our filters specify vans, and the time period, (and alaso exclude some weird fuel types)
WHERE typeApproval = 'N1'
AND monthOfFirstRegistration < '2022-01-01'
AND monthOfFirstRegistration > '2016-12-31'
AND fuelType IN ('DIESEL', 'PETROL', 'ELECTRICITY', 'HYBRID ELECTRIC')
# The group by statement groups rows with identical fueltype and monthOfFirstRegistration values, and gives the COUNT() function some work to do
GROUP BY monthOfFirstRegistration, fuelType

In [65]:
# This operation sets the index (think of the names of the rows) as the monthOfFirstRegistration values

df_van.set_index('monthOfFirstRegistration', inplace=True)

In [ ]:
df_van.head(20)

### How can we visualise this data?

We'll use the seaborn library to visualise this data - it's quite snazzy and fairly easy to get to grips with

Matplotlib is a very popular library, and has some more flexibility than seaborn, but it's a bit more complicated to get to grips with.




#### 1. Monthly totals in a line plot

In [68]:
df_van_wide = df_van.pivot(columns='fuelType', values='total')

In [ ]:
df_van_wide.plot.line(figsize=(12, 8))

#### 2. Market share over time

In [44]:
# We'll do a couple of operations here in one go...
# DataFrame.div() will divide a DataFrame by the value inside the brackets.
# We divide our dataframe by df_van_wide.sum(axis=1) which means that each row (indicated by 'axis=1') is summed up, so every value in each column is divided by the sum of each row, giving us the percentage)
# Uncomment the line of code below and run this cell to see what this operation does. 

# df_van_wide.div(df_van_wide.sum(axis=1), axis=0)

In [61]:
df_van_wide_perc = df_van_wide.div(df_van_wide.sum(axis=1), axis=0)

In [ ]:
df_van_wide_perc.plot(figsize=(12, 8))